In [1]:
!pip install requests matplotlib pandas

# Sagaパターン テストパターン概要

## 📊 テストデータ構成

このノートブックでは、Sagaパターン（Choreography/Orchestration）のテスト環境を構築し、検証するためのテストパターンを説明します。

### データ構成の全体像

```
マスタデータ (5顧客 + 5書籍 + 在庫)
├── 顧客: 田中太郎, 佐藤花子, 鈴木次郎, 高橋美咲, 伊藤健太
├── 書籍: マイクロサービスアーキテクチャ, 分散システム設計, etc.
└── 在庫: 現実的な在庫数（在庫切れパターン含む）

テストデータ
├── 正常ケース: 成功注文 (order-success-001)
├── 異常ケース1: 在庫不足 (order-fail-stock)
├── 異常ケース2: 決済失敗 (order-fail-payment)
├── イベントデータ: Choreographyパターンフロー
└── Sagaデータ: Orchestrationパターンフロー
```

In [2]:

import requests
import time
import pandas as pd
import matplotlib.pyplot as plt
import json

# Base URLs for the services
BASE_URL_CHOREOGRAPHY = "http://localhost:8001"  # Order Service for Choreography
BASE_URL_ORCHESTRATION = "http://localhost:8005" # Saga Orchestrator for Orchestration

# --- Test Scenarios ---

def run_test(pattern, scenario_name, endpoint, payload):
    """Generic function to run a test case."""
    if pattern == "choreography":
        url = f"{BASE_URL_CHOREOGRAPHY}{endpoint}"
    elif pattern == "orchestration":
        url = f"{BASE_URL_ORCHESTRATION}{endpoint}"
    else:
        return {"status": "error", "message": "Invalid pattern", "response_time": 0}

    start_time = time.time()
    try:
        response = requests.post(url, json=payload)
        response_time = time.time() - start_time
        response.raise_for_status()  # Raise an exception for bad status codes
        return {
            "pattern": pattern,
            "scenario": scenario_name,
            "status": "success",
            "status_code": response.status_code,
            "response_time": response_time,
            "response_json": response.json()
        }
    except requests.exceptions.RequestException as e:
        response_time = time.time() - start_time
        return {
            "pattern": pattern,
            "scenario": scenario_name,
            "status": "failure",
            "status_code": e.response.status_code if e.response else None,
            "response_time": response_time,
            "error_message": str(e)
        }

# --- Choreography Test Cases ---

def test_choreography_success():
    """Successful order placement."""
    payload = {
        "customerId": "customer-001",
        "bookId": "book-123",
        "quantity": 1
    }
    return run_test("choreography", "Choreography Success", "/orders", payload)

def test_choreography_stock_failure():
    """Failure due to insufficient stock."""
    payload = {
        "customerId": "customer-002",
        "bookId": "book-456", # This book has 0 stock
        "quantity": 1
    }
    return run_test("choreography", "Choreography Stock Failure", "/orders", payload)

def test_choreography_payment_failure():
    """Failure due to payment issues (simulated)."""
    # This requires the payment service to have a way to simulate failure.
    # We'll assume a special customerId or amount triggers this.
    payload = {
        "customerId": "customer-004", # Assuming this customer has payment issues
        "bookId": "book-789",
        "quantity": 1
    }
    return run_test("choreography", "Choreography Payment Failure", "/orders", payload)


# --- Orchestration Test Cases ---

def test_orchestration_success():
    """Successful order placement."""
    payload = {
        "customerId": "customer-001",
        "bookId": "book-123",
        "quantity": 1
    }
    return run_test("orchestration", "Orchestration Success", "/orders", payload)

def test_orchestration_stock_failure():
    """Failure due to insufficient stock."""
    payload = {
        "customerId": "customer-003",
        "bookId": "book-456", # This book has 0 stock
        "quantity": 1
    }
    return run_test("orchestration", "Orchestration Stock Failure", "/orders", payload)

def test_orchestration_payment_failure():
    """Failure due to payment issues (simulated)."""
    payload = {
        "customerId": "customer-004", # Assuming this customer has payment issues
        "bookId": "book-789",
        "quantity": 1
    }
    return run_test("orchestration", "Orchestration Payment Failure", "/orders", payload)

# --- Running all tests ---
results = []
results.append(test_choreography_success())
results.append(test_choreography_stock_failure())
results.append(test_choreography_payment_failure())
results.append(test_orchestration_success())
results.append(test_orchestration_stock_failure())
results.append(test_orchestration_payment_failure())

# Display results
results_df = pd.DataFrame(results)
print(results_df)


         pattern                       scenario   status status_code  \
0   choreography           Choreography Success  failure        None   
1   choreography     Choreography Stock Failure  failure        None   
2   choreography   Choreography Payment Failure  failure        None   
3  orchestration          Orchestration Success  failure        None   
4  orchestration    Orchestration Stock Failure  failure        None   
5  orchestration  Orchestration Payment Failure  failure        None   

   response_time                                      error_message  
0       0.039098  500 Server Error: Internal Server Error for ur...  
1       0.002416  500 Server Error: Internal Server Error for ur...  
2       0.001920  500 Server Error: Internal Server Error for ur...  
3       0.003739  404 Client Error: Not Found for url: http://lo...  
4       0.001001  404 Client Error: Not Found for url: http://lo...  
5       0.000875  404 Client Error: Not Found for url: http://lo...  


In [ ]:

# --- Visualization ---
def plot_results(df):
    """Plot test results."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

    # --- Bar chart for status ---
    status_counts = df.groupby(['pattern', 'status']).size().unstack(fill_value=0)
    status_counts.plot(kind='bar', ax=ax1, colormap='viridis', rot=0)
    ax1.set_title('Test Status by Saga Pattern')
    ax1.set_xlabel('Saga Pattern')
    ax1.set_ylabel('Number of Tests')
    ax1.legend(title='Status')

    # --- Bar chart for response times ---
    df_success = df[df['status'] == 'success']
    df_success.plot(kind='bar', x='scenario', y='response_time', ax=ax2, colormap='plasma')
    ax2.set_title('Response Time for Successful Tests')
    ax2.set_xlabel('Test Scenario')
    ax2.set_ylabel('Response Time (seconds)')
    ax2.tick_params(axis='x', rotation=45)

    plt.tight_layout()
    plt.show()

plot_results(results_df)
